In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

In [2]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [3]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': 'E:\\stud…

Button(description='ОК', style=ButtonStyle())

Output()

In [4]:
datasets = []

for fn in res.options:

    if res.options[fn] not in res.value:
        continue
    dataset = dp.Dataset(pd.read_csv(res.options[fn]), name=fn)

    dataset.target_features = ["Cover_Type"]
    datasets.append(dataset)

In [5]:
for d in datasets:
    print(d.name)



df_enhanced.csv_selected_features.csv_scaled.csv


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Input, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

def create_mlp_model(input_shape, num_classes, optimizer='adam', learning_rate=0.001, dense_units=[128, 64]):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    if optimizer == 'sgd': # learning_rate * grad(ФУНКЦИЯ ПОТЕРЬ)
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop': # градиенты нормализуются путем деления их на среднеквадратичное значение предыдущих
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_1d_cnn_model(input_shape, num_classes, filters=[128, 256],
                         kernel_size=3, pool_size=2, use_batch_norm=True, dense_units=[128, 64],
                         optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(input_shape, 1))) # input_shape уже должен быть (num_features, 1)
    i=0
    for f in filters:
        model.add(Conv1D(filters=f, kernel_size=kernel_size, activation='relu', padding='same'))
        #
        if use_batch_norm:
            model.add(BatchNormalization())
        # Добавляем MaxPooling1D, если есть смысл и достаточное количество признаков
        if input_shape > pool_size * (i + 1) : # Убедимся, что размерность не станет слишком маленькой
            model.add(MaxPooling1D(pool_size=pool_size))
        else:
            # Если MaxPooling1D может сделать размерность 0, пропускаем его
            # или можно настроить логику по-другому, например, убрать padding='same'
            # и добавить его только на первом слое
            pass # Пропускаем MaxPooling, если он приведет к 0 размерности
        i+=1
    model.add(Flatten())
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
        if use_batch_norm: # Можно добавить BatchNormalization и здесь, но это опционально
            model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))

    # Выбор оптимизатора
    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate) # Default

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


    

In [7]:
from itertools import product

def all_combinations(args_dict):
    # Получаем имена аргументов и их возможные значения
    arg_names = args_dict.keys()
    values = args_dict.values()
    res = []
    # Перебираем все комбинации значений
    for combination in product(*values):
        # Создаем словарь с аргументами
        kwargs = dict(zip(arg_names, combination))
        # Вызываем функцию с текущей комбинацией
        res.append(kwargs)
    return res



In [8]:

from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

for d in datasets:
    d.target_features
    temp_name = d.name
    num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов
    assert num_classes == 7
    temp = []
    for i in range(num_classes):
        temp.append(d.target_features[0] + f"_{i}")
    #d.target_features = 
    d[temp] = to_categorical(d[d.target_features].values - 1, num_classes=7) # Эта строка остается
    d = d.drop(d.target_features, axis=1)
    d = dp.Dataset(d)
    d.target_features = temp
    d.name = temp_name
    
    d.split_data(random_state=1337)

    all_models = []
    params = {"optimizer" : ['sgd', 'adam', 'rmsprop'], "learning_rate" : [0.00001, 0.0001], "dense_units":[[128, 64, 32], [128, 64], [128], [64]]}
    params_1d_cnn = {"kernel_size" : [3], "pool_size": [2], "use_batch_norm": [False, True], "filters":[[128, 256], [64, 128, 256]]}
    params_mlp = {}
    params_default = {"input_shape": [d.shape[1] - num_classes], "num_classes" : [num_classes]}
    
    
    mlp_comb = all_combinations({**params_default, **params, **params_mlp})
    cnn_comb = all_combinations({**params_default, **params, **params_1d_cnn})
    #print(len(mlp_comb))
    print(len(cnn_comb))

    iii = 0
    #for el in mlp_comb:
    for el in cnn_comb:

        print(iii, el)
        iii+=1
        model = create_mlp_model(**mlp_comb[0])
        model.summary()

        model = create_1d_cnn_model(**el)
        model.summary()

        break
        history = model.fit(d.X_train.values, d.Y_train.values, epochs=20, batch_size=64, callbacks=[early_stopping])
        with open('results3.txt', 'a', encoding='utf-8') as f:
            f.write(f"\n\n{d.name}; {el};")

        

        Y_train_real = model.predict(d.X_train.values, verbose=0)
        Y_test_real = model.predict(d.X_test.values, verbose=0)
        Y_valid_real = model.predict(d.X_valid.values, verbose=0)
       # print(np.isnan(Y_train_real).any())
       # print(np.isnan(Y_test_real).any())
       # print(np.isnan(Y_valid_real).any())

        AAA = [[np.argmax(Y_train_real, axis=1), np.argmax(d.Y_train.values, axis=1), "train"], [np.argmax(Y_test_real, axis=1), np.argmax(d.Y_test.values, axis=1), "test"], [np.argmax(Y_valid_real, axis=1), np.argmax(d.Y_valid.values, axis=1), "valid"]]
        with open('results3.txt', 'a', encoding='utf-8') as f:
            for viborka in AAA:
                try:
                    #print(type(viborka[0]), type(viborka[1]))
                    #print(accuracy_score(viborka[0], viborka[1]))
                    f.write(f"{viborka[2]}; acc= {accuracy_score(viborka[0], viborka[1])};\n")
                except Exception as e:
                    print(e)
                    f.write(f"{viborka[2]}; acc= accuracy_score_error;\n")
                try:

                    f.write(f"{viborka[2]}; bal_acc= {balanced_accuracy_score(viborka[0], viborka[1])};\n")
                except:
                    f.write(f"{viborka[2]}; bal_acc= balanced_accuracy_score_error;\n")

                try:

                    f.write(f"{viborka[2]}; f1_macro= {f1_score(viborka[0], viborka[1], average='macro', zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; f1_macro= f1_score_error;\n")

                try:

                    f.write(f"{viborka[2]}; f1_weighted= {f1_score(viborka[0], viborka[1], average='weighted', zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; f1_weighted= f1_score_error;\n")

                try:
                    f.write(f"{viborka[2]}; classification_report= {classification_report(viborka[0], viborka[1], target_names=[f'Class {i+1}' for i in range(num_classes)], zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; classification_report= classification_report_error;\n")

    


C:\Users\nikol\AppData\Local\Temp\ipykernel_11408\675742459.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов


96
0 {'input_shape': 12, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64, 32], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': False, 'filters': [128, 256]}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             231 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,231 (47.78 KB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 12, 128)             │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 6, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 6, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 3, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 7)                   │             231 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 208,071 (812.78 KB)

 Trainable params: 208,071 (812.78 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]
